<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#主题模型-以微博文本为例" data-toc-modified-id="主题模型-以微博文本为例-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>主题模型-以微博文本为例</a></span><ul class="toc-item"><li><span><a href="#文本预处理" data-toc-modified-id="文本预处理-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>文本预处理</a></span></li><li><span><a href="#主题建模" data-toc-modified-id="主题建模-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>主题建模</a></span></li></ul></li><li><span><a href="#主题模型-以外交部记者会数据为例" data-toc-modified-id="主题模型-以外交部记者会数据为例-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>主题模型-以外交部记者会数据为例</a></span><ul class="toc-item"><li><span><a href="#媒体类型编码" data-toc-modified-id="媒体类型编码-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>媒体类型编码</a></span></li><li><span><a href="#预处理" data-toc-modified-id="预处理-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>预处理</a></span></li><li><span><a href="#LDA模型1---外交部发言人回答" data-toc-modified-id="LDA模型1---外交部发言人回答-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>LDA模型1 - 外交部发言人回答</a></span></li><li><span><a href="#LDA模型2---港澳台·国外媒体提问" data-toc-modified-id="LDA模型2---港澳台·国外媒体提问-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>LDA模型2 - 港澳台·国外媒体提问</a></span></li><li><span><a href="#LDA模型2---中国大陆媒体提问" data-toc-modified-id="LDA模型2---中国大陆媒体提问-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>LDA模型2 - 中国大陆媒体提问</a></span></li></ul></li></ul></div>

# 主题模型-以微博文本为例

In [42]:
import pandas as pd
import numpy as np
import re
import jieba
import matplotlib.pyplot as plt
from gensim import corpora, models #从Gensim库中导入corpora和models模块，用于构建词袋和执行LDA主题建模
import pyLDAvis #用于将主题模型的结果以交互可视化的形式展现出来
import pyLDAvis.gensim_models as gensimvis

In [43]:
df = pd.read_excel('text_analysis_weibo_sample.xlsx', index_col = 0)

In [44]:
df.head()

,index,标题/微博内容,点赞,转发,评论,账号昵称UID加密,粉丝数,关注数,地域
0,34121,国债：地产行业重磅利好提振风险偏好，期债低开低走 国债期货全线收跌，10年期主力...,0,0,0,e5df796860e68f403bcf9651bab4d42e,0,0,其他
1,40230,#喜迎二十大 忠诚保平安#,0,0,0,6e35cb69ad52f20de5e28197b2e85306,405444,252,广西
2,7714,注意！事关明日教资考试！福建省教育考试院发布补充公告 福建省2022年下半年全国中小学教师...,0,0,0,e6953217442e6c06a7af23eee5e185f2,53264,2177,福建
3,27378,近日，“千年大计”雄安新区迎来五周岁生日。从“一张白纸...,0,0,0,NaN,0,0,北京
4,15435,樊振东牛逼！,0,0,0,344af41eac516375c04dee6325e763cc,8,51,山东


## 文本预处理
1. 清除文本中非中文字符，如标点符号、数字、英语字母等<br>
2. 去除停用词，预先准备好停用词列表<br>
3. 中文分词

In [45]:
def remove_nums(text): #定义一个去掉文本中数字（只保留中文字符）的函数
    nonums = re.sub('[^\u4e00-\u9fa5]+', '', text) #Unicode范围为4E00到9FA5的是中文汉字，这里去除掉所有非中文内容
    return nonums
test = df['标题/微博内容'][0] #测试函数
remove_nums(test)[:100] #将函数功能应用于表格的前100行

'国债地产行业重磅利好提振风险偏好期债低开低走国债期货全线收跌年期主力合约跌年期主力合约跌年期主力合约跌三大主力合约均创逾一年收盘新低行情解读公开市场方面央行开展了亿元天期逆回购操作中标利率亿元逆回购到'

In [46]:
stopwords = open('stopwords.txt', encoding = 'utf-8').read()
def clean_text(text):
    words = jieba.lcut(text)
    words = [w for w in words if w not in stopwords and w!='[' and w!=']'] #去除停用词
    return ' '.join(words)

In [47]:
test = df['标题/微博内容'][0]
clean_text(test)

'国债 地产 行业 重磅 利好 提振 风险 偏好 期债 低开 低 走 国债 期货 全线 收跌 10 年期 主力 合约 跌 0.37% 5 年期 主力 合约 跌 0.24% 年期 主力 合约 跌 0.11% 三大 主力 合约 均 创 逾 一年 收盘 新低 行情 解读 公开市场 央行 800 亿元 7 天期 逆 回购 操作 中标 利率 2.0% 亿元 逆 回购 到期 当日 净 投放 780 亿元 资金面 央行 公开市场 逆 回购 加力 银行 市场 周二 资金 供给 趋于 收敛 七天 回购 加权 利率 进一步 上行 长期 资金 全国 股份制 银行 一年期 同业 存单 报价 2.45% 水平 尚无 成交量 配合 3 房地产 行业 再度 迎来 重磅 利好 证监会 涉房 企业 股权 融资 调整 优化 五项 措施 自即日起 施行 证监会 支持 房企 股权 融资 第三支 箭 正式 发射 信贷 债券 股权 三个 融资 渠道 三箭 齐发 合力 推动 房地产 融资 第三支 箭 快速 落地 房地产 行业 注入 大规模 增量 资金 资产 行业 平稳 健康 发展 短短 时间 政策 端 三箭 齐发 强势 托底 提振 各方 信心 看好 行业 信用 复苏 因城 施策 空间 打开 销售 回暖 4 国务院 联防 联控 机制 11 月 29 日 15 新闻 发布会 针对 地方 不满情绪 政府 重新考虑 疫情 应对 政策 国家 卫生 健康 委 新闻 发言人 宣传司 副司长 米锋 称 疫情 发生 中国政府 病毒 变异 临床 治疗 实践 防控 研究 调整 针对 奥密克戎 变异 株 传播 致病 特性 重症 率 死亡率 情况 包括 关注 国际 疫情 形势 变化 防控 措施 研究 调整 程度 保护 人民 利益 限度 减少 疫情 经济社会 发展 影响 5 近期 政策 端 房地产 金融 支持 稳 地产 宽 信用 绑定 房地产业 关联 很多 上下游 行业 良性循环 经济 健康 发展 意义 年末 稳 增长 诉求 提升 央行 降准 助力 稳 增长 熨平 流动性 压力 支持 信贷 投放 综合 融资 成本 稳中有降 落实 稳 经济 一揽子 政策措施 经济 回稳 向上 基础 提振 市场 风险 偏好 股市 大涨 债市 承压 市场 资金 长期 预期 暂 不明朗 短期内 看空 气氛 略重 操作 建议 三大期 债 主力 连续 加权 合约 反

In [48]:
df['标题/微博内容'] = df['标题/微博内容'].astype(str) #将标题/微博内容中的对象转化为字符串类型
#在dataframe中添加“微博内容分词”一列
df['微博内容分词'] = df['标题/微博内容'].apply(remove_nums) #执行清洗操作
df['微博内容分词'] = df['微博内容分词'].apply(clean_text) #执行分词操作
df['微博内容分词'] = df['微博内容分词'].apply(lambda x: x.split()) #将由空格分割的字符串转化为列表
df

,index,标题/微博内容,点赞,转发,评论,账号昵称UID加密,粉丝数,关注数,地域,微博内容分词
0,34121,国债：地产行业重磅利好提振风险偏好，期债低开低走 国债期货全线收跌，10年期主力...,0,0,0,e5df796860e68f403bcf9651bab4d42e,0,0,其他,"[国债, 地产, 行业, 重磅, 利好, 提振, 风险, 偏好, 期债, 低开, 低, 走,..."
1,40230,#喜迎二十大 忠诚保平安#,0,0,0,6e35cb69ad52f20de5e28197b2e85306,405444,252,广西,"[喜迎, 二十大, 忠诚, 保平安]"
2,7714,注意！事关明日教资考试！福建省教育考试院发布补充公告 福建省2022年下半年全国中小学教师...,0,0,0,e6953217442e6c06a7af23eee5e185f2,53264,2177,福建,"[事关, 明日, 教资, 考试, 福建省, 教育, 考试院, 发布, 补充, 公告, 福建省..."
3,27378,近日，“千年大计”雄安新区迎来五周岁生日。从“一张白纸...,0,0,0,NaN,0,0,北京,"[近日, 千年, 大计, 雄安, 新区, 迎来, 五周岁, 生日, 一张白纸, 塔吊, 林立..."
4,15435,樊振东牛逼！,0,0,0,344af41eac516375c04dee6325e763cc,8,51,山东,"[樊振东, 牛, 逼]"
...,...,...,...,...,...,...,...,...,...,...
95,14034,老公好漂亮[舔屏],0,0,0,aa7af97ae98cbe8983e559b948ecfabf,825,191,北京,"[老公, 漂亮, 舔, 屏]"
96,35249,恭喜@张小娜呀娜 1名用户获得【小飞T】。C官方唯一抽奖工具@C抽奖平台 对本次抽奖进行监督...,0,0,2,f538513e5801c275cbcf285517a8ee62,51709,20,北京,"[恭喜, 张小娜, 娜, 名, 用户, 小飞, 官方, 唯一, 抽奖, 工具, 抽奖, 平台..."
97,39598,接不动了，居民部门负债率提升空间太有限了。现在动辄几万一平米，总价高，月供高，预期转差，还怎...,0,0,0,2e76c39b6665a0b6f5bdc4c35f36f573,0,59,四川,"[接不动, 居民, 部门, 负债率, 提升, 空间, 有限, 动辄, 几万, 平米, 总价,..."
98,48977,目前电子烟政策已从制定阶段逐渐走向实施阶段，而主要的电子烟公司股价下跌幅度高达70%-...,0,0,0,NaN,0,0,北京,"[电子, 烟, 政策, 制定, 阶段, 走向, 实施, 阶段, 电子, 烟, 股价, 下跌,..."


## 主题建模
1. 构建语料库<br>
2. 创建LDA主题模型，使用语料库进行训练<br>
3. 打印每个主题的特征词<br>
4. 查看每个文档的主题分布，即判定某文档应该被归为哪个主题<br>
5. 可视化显示LDA结果

In [49]:
dictionary = corpora.Dictionary(df['微博内容分词']) #使用gensim的corpora构建一个词典 
corpus = [dictionary.doc2bow(text) for text in df['微博内容分词']] #遍历文本并转化为词袋

In [50]:
lda_model = models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)
#指定识别出5个主题，将词ID映射到词本身，训练15遍

In [51]:
topics = lda_model.print_topics(num_words=5) #从每个主题中提取权重（类似TF-IDF）前5的词
for topic in topics:
    print(topic)

(0, '0.038*"电子" + 0.038*"烟" + 0.011*"中国" + 0.010*"政策" + 0.008*"市场"')
(1, '0.013*"郑州" + 0.012*"亿元" + 0.010*"重庆" + 0.006*"城市" + 0.006*"银行"')
(2, '0.021*"月" + 0.015*"万吨" + 0.012*"市场" + 0.010*"日" + 0.009*"库存"')
(3, '0.011*"数字" + 0.009*"中国" + 0.008*"经济" + 0.008*"建设" + 0.007*"发展"')
(4, '0.021*"考生" + 0.013*"考试" + 0.011*"考场" + 0.009*"刘雨昕" + 0.008*"考点"')


In [52]:
df['微博内容分词'].iloc[0] #获取分词结果第一行的内容

['国债',
 '地产',
 '行业',
 '重磅',
 '利好',
 '提振',
 '风险',
 '偏好',
 '期债',
 '低开',
 '低',
 '走',
 '国债',
 '期货',
 '全线',
 '收跌',
 '年期',
 '主力',
 '合约',
 '跌',
 '年期',
 '主力',
 '合约',
 '跌',
 '年期',
 '主力',
 '合约',
 '跌三大',
 '主力',
 '合约',
 '均',
 '创',
 '逾',
 '一年',
 '收盘',
 '新低',
 '行情',
 '解读',
 '公开市场',
 '央行',
 '亿元',
 '天期',
 '逆',
 '回购',
 '操作',
 '中标',
 '利率',
 '亿元',
 '逆',
 '回购',
 '到期',
 '当日',
 '净',
 '投放',
 '亿元',
 '资金面',
 '央行',
 '公开市场',
 '逆',
 '回购',
 '加力',
 '银行',
 '市场',
 '周二',
 '资金',
 '供给',
 '趋于',
 '收敛',
 '七天',
 '回购',
 '加权',
 '利率',
 '进一步',
 '上行',
 '长期',
 '资金',
 '全国',
 '股份制',
 '银行',
 '一年期',
 '同业',
 '存单',
 '报价',
 '水平',
 '尚无',
 '成交量',
 '配合',
 '房地产',
 '行业',
 '再度',
 '迎来',
 '重磅',
 '利好',
 '证监会',
 '涉房',
 '企业',
 '股权',
 '融资',
 '调整',
 '优化',
 '五项',
 '措施',
 '自即日起',
 '施行',
 '证监会',
 '支持',
 '房企',
 '股权',
 '融资',
 '第三支',
 '箭',
 '正式',
 '发射',
 '信贷',
 '债券',
 '股权',
 '三个',
 '融资',
 '渠道',
 '三箭',
 '齐发',
 '合力',
 '推动',
 '房地产',
 '融资',
 '第三支',
 '箭',
 '快速',
 '落地',
 '房地产',
 '行业',
 '注入',
 '大规模',
 '增量',
 '资金',
 '资产',
 '行业',
 '平稳',
 '健康',
 '发展',
 '短短',
 '时间',

In [53]:
for index, score in sorted(lda_model[corpus[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))
#显示了第一个文档（第一行微博）在特定主题上的分布概率，以及主题分布列表，从结果可以看出，该文档属于主题3（第4个主题）


Score: 0.999202311038971	 
Topic: 0.021*"月" + 0.015*"万吨" + 0.012*"市场" + 0.010*"日" + 0.009*"库存" + 0.008*"预期" + 0.007*"需求" + 0.006*"下降" + 0.006*"环比" + 0.006*"价格"


In [54]:
documents = df['微博内容分词'].values

def infer_topics(lda_model, document): #定义推断文档主题的函数，返回文档中主题的概率分布
    bow = dictionary.doc2bow(document)
    topics = lda_model.get_document_topics(bow)
    return topics

for i, doc in enumerate(documents[:10]): #对前10个文档使用上述主题推断函数
    doc_topics = infer_topics(lda_model, doc)
    print(f"Document {i+1}:")
    print(doc_topics)
    print()

Document 1:
[(2, 0.9992023)]

Document 2:
[(0, 0.04003362), (1, 0.83988154), (2, 0.040023472), (3, 0.04002744), (4, 0.04003392)]

Document 3:
[(4, 0.99647343)]

Document 4:
[(3, 0.9990871)]

Document 5:
[(0, 0.050041668), (1, 0.05030928), (2, 0.050032843), (3, 0.79957414), (4, 0.050042048)]

Document 6:
[(0, 0.040113315), (1, 0.040096488), (2, 0.83958346), (3, 0.0400924), (4, 0.040114336)]

Document 7:
[(0, 0.016693773), (1, 0.016762929), (2, 0.016767053), (3, 0.93303007), (4, 0.016746175)]

Document 8:
[(0, 0.100006565), (1, 0.10033877), (2, 0.59741306), (3, 0.10001014), (4, 0.10223145)]

Document 9:
[(0, 0.94251376), (1, 0.014395431), (2, 0.014328187), (3, 0.014411644), (4, 0.014350997)]

Document 10:
[(1, 0.998903)]



In [55]:
#对LDA主题模型进行可视化
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary, n_jobs=1) #n_jobs=1表示单线程处理
pyLDAvis.display(lda_vis)

左图（Intertopic Distance Map）表示每个主题间的距离，每个点的大小表示主题的重要性；右图显示每个主题中最重要的30个词

In [56]:
# 导出可视化结果到html
pyLDAvis.save_html(lda_vis, 'lda_visualization.html')

# 主题模型-以外交部记者会数据为例
**说明**<br>
从外交部网站爬取的文档以每场记者会为单位，而非每个问答。将每场包含多个问答的记者会作为主题模型的分析单元并不合适。考虑到将一篇记者会发言的文字稿拆分成多个问答的复杂性，以及现实中存在可利用资源的优势，因此，本次作业将使用已有的数据集——Chinese Ministry of Foreign Affairs Press Conferences Corpus, CMFA PressCon[1]。<br>
该数据集搜集了从2002年10月15日至2023年12月31日的期间的所有外交部记者会问答的数据，并举行了结构化处理，包括问题、回答、经过Lemmatization的问题、回答、发言人、时间、位置实体、组织实体、个人实体等多列内容，文本语言为英语。<br>
<br>
**研究问题**：中国大陆媒体、外国·港澳台媒体、中方问答三部分文本中呈现的主题有何差异？<br>
**研究步骤**
1. 提取包含提问对象的文本内容，外交部网站中公示的记者会发言文本从2020年3月24日起加入提问媒体单位数据。
2. 提取原结构化数据中的who_asked一列中包含的所有媒体单位，人工编码确定媒体属于中方媒体或外方媒体。
3. 分别进行预处理，识别出中国大陆媒体提问、外国·港澳台媒体提问、记者会发言人回答三部分子文本的主题。

[1]Mochtak, Michal and Richard Q. Turcsanyi (2021): "Studying Chinese Foreign Policy Narratives: Introducing the Ministry of Foreign Affairs Press Conferences Corpus". Journal of Chinese Political Science, 26 (4): 743-761. doi: 10.1007/s11366-021-09762-3

## 媒体类型编码
1. 取总数据集中2020年3月24日及以后的数据，从该日起，外交部记者会开始记录提问记者媒体单位
2. 使用Excel中筛选功能得出所有媒体单位数据，人工编码（1代表中国大陆媒体、2代表港澳台·国外媒体、3代表其他），得出编码手册
3. 按照人工编码在数据集中添加“Media Type”列，得出新数据集“CMFA_Dataset.xlsx”

In [57]:
import pandas as pd

dataset_path = 'CMFA Dataset.xlsx'
coding_manual_path = '编码手册.xlsx'

dataset = pd.read_excel(dataset_path)
coding_manual = pd.read_excel(coding_manual_path)

media_type_dict = dict(zip(coding_manual['Media Name'], coding_manual['Type']))

def get_media_type(media_name):
    return media_type_dict.get(media_name, 'Unknown')

dataset['Media Type'] = dataset['who_asked'].apply(get_media_type)

output_path = 'CMFA_Dataset.xlsx'
dataset.to_excel(output_path, index=False)

/var/folders/qy/l6pd6hxj5gn51xzk2nltjm2h0000gn/T/ipykernel_56861/4070338159.py:17: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  dataset.to_excel(output_path, index=False)


In [58]:
# 提取出媒体类型为1和2的行，以及用于后续分析的列
file_path = 'CMFA_Dataset.xlsx' 
df = pd.read_excel(file_path)

filtered_df = df[df['Media Type'].isin([1, 2])]

columns_to_extract = ['id_orig', 'date', 'question_lem', 'answer_lem', 'who_asked', 'Media Type']
washed_df = filtered_df[columns_to_extract]

In [59]:
washed_df.head()

,id_orig,date,question_lem,answer_lem,who_asked,Media Type
0,19174,2020-03-24,last Friday you announce that China and latin ...,"on March 24, expert on COVID- 19 from China an...",CCTV,1
1,19175,2020-03-24,in a interview with the chinese ambassador to ...,we state China 's position for many time. the ...,The Paper,1
2,19176,2020-03-24,"the New York Times, Washington Post and Wall S...",I haven't read this letter yet. but as I recal...,Reuters,2
3,19177,2020-03-24,"China have deplore and reject the phrase"" chin...",the chinese side have repeat many time that so...,China Daily,1
4,19178,2020-03-24,you talk about the import case yesterday. coul...,journalist with the Press trust of India ask a...,CRI,1


In [60]:
len(washed_df)

9751

## 预处理
1. 已经经过了Lemmatization处理
2. 小写
3. 去除掉数字
4. 去除掉标点符号
5. 去除掉非英文内容和停用词
6. 使用空格分割符生成列表，元素为单词

In [61]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) #去除英文停用词
stop_words.add('china') #添加china作为停用词，提升模型性能

def preprocess_text(text):
    text = text.lower() #小写
    text = re.sub(r'\d+', '', text) #删除数字
    text = re.sub(r'[^\w\s]', '', text) #删除标点符号
    words = text.split() #用空格分割
    words = [word for word in words if word not in stop_words and re.match(r'[a-zA-Z]+', word)]  # 去除停用词和非字母单词
    text = ' '.join(words)
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/liuzhixin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [62]:
washed_df['question_lem'] = washed_df['question_lem'].astype(str) #将对象转化为字符串类型
washed_df['answer_lem'] = washed_df['answer_lem'].astype(str)
#在dataframe中添加“question_lem_new”和“answer_lem_new”列，用于储存预处理结果
washed_df['question_lem_new'] = washed_df['question_lem'].apply(preprocess_text) #执行清洗操作
washed_df['answer_lem_new'] = washed_df['answer_lem'].apply(preprocess_text) 
washed_df['question_lem_new'] = washed_df['question_lem_new'].apply(lambda x: x.split()) #将由空格分割的字符串转化为列表
washed_df['answer_lem_new'] = washed_df['answer_lem_new'].apply(lambda x: x.split())
washed_df.head()

/var/folders/qy/l6pd6hxj5gn51xzk2nltjm2h0000gn/T/ipykernel_56861/2806956545.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  washed_df['question_lem'] = washed_df['question_lem'].astype(str) #将对象转化为字符串类型
/var/folders/qy/l6pd6hxj5gn51xzk2nltjm2h0000gn/T/ipykernel_56861/2806956545.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  washed_df['answer_lem'] = washed_df['answer_lem'].astype(str)
/var/folders/qy/l6pd6hxj5gn51xzk2nltjm2h0000gn/T/ipykernel_56861/2806956545.py:4: SettingWithCopyWarning: 
A value 

,id_orig,date,question_lem,answer_lem,who_asked,Media Type,question_lem_new,answer_lem_new
0,19174,2020-03-24,last Friday you announce that China and latin ...,"on March 24, expert on COVID- 19 from China an...",CCTV,1,"[last, friday, announce, latin, american, cari...","[march, expert, covid, latin, american, caribb..."
1,19175,2020-03-24,in a interview with the chinese ambassador to ...,we state China 's position for many time. the ...,The Paper,1,"[interview, chinese, ambassador, us, american,...","[state, position, many, time, measure, take, c..."
2,19176,2020-03-24,"the New York Times, Washington Post and Wall S...",I haven't read this letter yet. but as I recal...,Reuters,2,"[new, york, times, washington, post, wall, str...","[havent, read, letter, yet, recall, make, clea..."
3,19177,2020-03-24,"China have deplore and reject the phrase"" chin...",the chinese side have repeat many time that so...,China Daily,1,"[deplore, reject, phrase, chinese, virus, wuha...","[chinese, side, repeat, many, time, people, us..."
4,19178,2020-03-24,you talk about the import case yesterday. coul...,journalist with the Press trust of India ask a...,CRI,1,"[talk, import, case, yesterday, could, tell, l...","[journalist, press, trust, india, ask, yesterd..."


## LDA模型1 - 外交部发言人回答

In [63]:
import gensim.corpora as corpora
from gensim.models import LdaModel
import pyLDAvis 
import pyLDAvis.gensim_models as gensimvis

In [64]:
texts = washed_df['answer_lem_new'].tolist()
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [65]:
lda_model = LdaModel(corpus, num_topics= 8, id2word=dictionary, passes=15)  # 训练LDA模型

In [66]:
# 输出每个主题的词语分布
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.049*"taiwan" + 0.029*"us" + 0.024*"one" + 0.018*"principle" + 0.015*"sovereignty" + 0.012*"independence" + 0.012*"relation" + 0.011*"international" + 0.010*"question" + 0.009*"territorial"')
(1, '0.053*"us" + 0.029*"country" + 0.015*"world" + 0.013*"international" + 0.012*"cooperation" + 0.010*"development" + 0.009*"war" + 0.008*"peace" + 0.008*"interest" + 0.007*"security"')
(2, '0.017*"peace" + 0.017*"issue" + 0.017*"security" + 0.014*"party" + 0.014*"international" + 0.011*"ukraine" + 0.009*"situation" + 0.009*"un" + 0.009*"afghanistan" + 0.009*"political"')
(3, '0.035*"nuclear" + 0.034*"japan" + 0.021*"international" + 0.017*"water" + 0.013*"japanese" + 0.012*"discharge" + 0.010*"contaminate" + 0.009*"ocean" + 0.009*"community" + 0.009*"concern"')
(4, '0.023*"chinese" + 0.023*"law" + 0.018*"us" + 0.016*"hong" + 0.016*"kong" + 0.012*"affair" + 0.012*"company" + 0.011*"interest" + 0.011*"side" + 0.011*"security"')
(5, '0.018*"chinese" + 0.016*"country" + 0.015*"covid" + 0.010*

In [67]:
washed_df['answer_lem_new'].iloc[0]

['march',
 'expert',
 'covid',
 'latin',
 'american',
 'caribbean',
 'country',
 'exchange',
 'view',
 'video',
 'conference',
 'official',
 'expert',
 'latin',
 'american',
 'caribbean',
 'country',
 'diplomatic',
 'relation',
 'nicaragua',
 'well',
 'representative',
 'unicef',
 'pan',
 'american',
 'health',
 'organization',
 'interamerican',
 'development',
 'bank',
 'attend',
 'video',
 'conference',
 'last',
 'three',
 'hour',
 'achieve',
 'positive',
 'result',
 'chinese',
 'expert',
 'talk',
 'detail',
 'trajectory',
 'feature',
 'epidemic',
 'chinese',
 'experience',
 'disease',
 'control',
 'clinical',
 'treatment',
 'custom',
 'administration',
 'community',
 'base',
 'quarantine',
 'give',
 'thorough',
 'answer',
 'foreign',
 'counterpart',
 'question',
 'foreign',
 'official',
 'expert',
 'speak',
 'highly',
 'effective',
 'measure',
 'contain',
 'virus',
 'important',
 'contribution',
 'global',
 'public',
 'health',
 'security',
 'believe',
 'timely',
 'professional',
 '

In [68]:
for index, score in sorted(lda_model[corpus[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))
#第一行的内容可以归在新冠这个类别里


Score: 0.671105682849884	 
Topic: 0.018*"chinese" + 0.016*"country" + 0.015*"covid" + 0.010*"vaccine" + 0.009*"information" + 0.009*"international" + 0.008*"health" + 0.008*"provide" + 0.007*"relevant" + 0.007*"global"

Score: 0.32297736406326294	 
Topic: 0.025*"cooperation" + 0.018*"country" + 0.018*"development" + 0.011*"two" + 0.009*"new" + 0.009*"president" + 0.009*"year" + 0.009*"relation" + 0.009*"global" + 0.008*"world"


In [69]:
documents = washed_df['answer_lem_new'].values
def infer_topics(lda_model, document):
    bow = dictionary.doc2bow(document)
    topics = lda_model.get_document_topics(bow)
    return topics

for i, doc in enumerate(documents[:10]):
    doc_topics = infer_topics(lda_model, doc)
    print(f"Document {i+1}:")
    print(doc_topics)
    print()

Document 1:
[(5, 0.671099), (6, 0.322984)]

Document 2:
[(4, 0.34507385), (5, 0.033684723), (7, 0.61780214)]

Document 3:
[(2, 0.07669038), (4, 0.5289819), (7, 0.38438395)]

Document 4:
[(1, 0.2696245), (4, 0.02830142), (5, 0.37254217), (6, 0.028525813), (7, 0.29950717)]

Document 5:
[(3, 0.07434056), (5, 0.797282), (7, 0.10517944)]

Document 6:
[(2, 0.06058735), (4, 0.09366885), (5, 0.6731885), (7, 0.16420029)]

Document 7:
[(0, 0.52639294), (1, 0.17564966), (3, 0.18882978), (4, 0.09774449)]

Document 8:
[(4, 0.5432087), (5, 0.39546844), (7, 0.036291193)]

Document 9:
[(0, 0.01390595), (1, 0.013902395), (2, 0.0139012635), (3, 0.013903749), (4, 0.90267754), (5, 0.013910819), (6, 0.013893477), (7, 0.013904788)]

Document 10:
[(0, 0.59094054), (2, 0.24534121), (5, 0.14530997)]



In [70]:
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary, n_jobs=1) 
pyLDAvis.display(lda_vis)
#主题1:中国外交动作，主题2:新疆问题，主题3:抗击新冠，主题4:战争与军事，主题5:香港问题，主题6:地区局势，主题7:台湾问题，主题8:日本排放核污水

## LDA模型2 - 港澳台·国外媒体提问

In [71]:
df_media_type_2 = washed_df[washed_df['Media Type'] == 2]
df_media_type_2.head()
len(df_media_type_2)

5729

In [72]:
texts = df_media_type_2['answer_lem_new'].tolist()
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [73]:
lda_model = LdaModel(corpus, num_topics= 8, id2word=dictionary, passes=15) 

In [74]:
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.024*"kong" + 0.024*"hong" + 0.020*"xinjiang" + 0.017*"affair" + 0.017*"law" + 0.017*"rights" + 0.012*"people" + 0.012*"country" + 0.012*"human" + 0.011*"internal"')
(1, '0.029*"cooperation" + 0.023*"country" + 0.017*"relation" + 0.017*"development" + 0.015*"two" + 0.011*"mutual" + 0.009*"side" + 0.009*"bilateral" + 0.008*"work" + 0.008*"people"')
(2, '0.046*"taiwan" + 0.045*"us" + 0.022*"one" + 0.016*"principle" + 0.014*"relation" + 0.014*"sovereignty" + 0.011*"independence" + 0.011*"question" + 0.011*"side" + 0.010*"stop"')
(3, '0.063*"japan" + 0.027*"nuclear" + 0.023*"japanese" + 0.015*"water" + 0.013*"international" + 0.012*"discharge" + 0.012*"pakistan" + 0.011*"ocean" + 0.011*"contaminate" + 0.008*"neighbor"')
(4, '0.019*"issue" + 0.015*"party" + 0.015*"ukraine" + 0.014*"situation" + 0.014*"relevant" + 0.013*"position" + 0.013*"peace" + 0.012*"question" + 0.010*"side" + 0.009*"information"')
(5, '0.013*"nuclear" + 0.012*"olympic" + 0.011*"committee" + 0.010*"international" 

In [75]:
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary, n_jobs=1) 
pyLDAvis.display(lda_vis)

## LDA模型2 - 中国大陆媒体提问

In [76]:
df_media_type_1 = washed_df[washed_df['Media Type'] == 1]
df_media_type_1.head()

,id_orig,date,question_lem,answer_lem,who_asked,Media Type,question_lem_new,answer_lem_new
0,19174,2020-03-24,last Friday you announce that China and latin ...,"on March 24, expert on COVID- 19 from China an...",CCTV,1,"[last, friday, announce, latin, american, cari...","[march, expert, covid, latin, american, caribb..."
1,19175,2020-03-24,in a interview with the chinese ambassador to ...,we state China 's position for many time. the ...,The Paper,1,"[interview, chinese, ambassador, us, american,...","[state, position, many, time, measure, take, c..."
3,19177,2020-03-24,"China have deplore and reject the phrase"" chin...",the chinese side have repeat many time that so...,China Daily,1,"[deplore, reject, phrase, chinese, virus, wuha...","[chinese, side, repeat, many, time, people, us..."
4,19178,2020-03-24,you talk about the import case yesterday. coul...,journalist with the Press trust of India ask a...,CRI,1,"[talk, import, case, yesterday, could, tell, l...","[journalist, press, trust, india, ask, yesterd..."
10,19185,2020-03-25,it be announce today that President Xi Jinping...,"currently, COVID- 19 be engulf many part of th...",CCTV,1,"[announce, today, president, xi, jinping, atte...","[currently, covid, engulf, many, part, world, ..."


In [77]:
len(df_media_type_1)

4022

In [78]:
texts = df_media_type_1['answer_lem_new'].tolist()
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [79]:
lda_model = LdaModel(corpus, num_topics= 8, id2word=dictionary, passes=15) 

In [80]:
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.037*"cooperation" + 0.026*"country" + 0.025*"road" + 0.022*"year" + 0.021*"green" + 0.019*"belt" + 0.016*"development" + 0.013*"bri" + 0.012*"international" + 0.012*"project"')
(1, '0.020*"country" + 0.015*"global" + 0.014*"development" + 0.014*"international" + 0.012*"community" + 0.012*"world" + 0.011*"cooperation" + 0.009*"people" + 0.008*"security" + 0.007*"share"')
(2, '0.045*"president" + 0.019*"xi" + 0.015*"minister" + 0.013*"country" + 0.013*"prime" + 0.010*"new" + 0.009*"leader" + 0.009*"various" + 0.009*"year" + 0.008*"relation"')
(3, '0.014*"security" + 0.014*"resolution" + 0.011*"peace" + 0.011*"afghanistan" + 0.011*"support" + 0.011*"country" + 0.010*"continue" + 0.010*"chinese" + 0.009*"humanitarian" + 0.009*"situation"')
(4, '0.029*"international" + 0.019*"japan" + 0.017*"nuclear" + 0.016*"law" + 0.014*"sovereignty" + 0.014*"water" + 0.013*"cable" + 0.013*"country" + 0.011*"sea" + 0.010*"community"')
(5, '0.047*"us" + 0.047*"cooperation" + 0.033*"space" + 0.025*"s

In [81]:
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary, n_jobs=1) 
pyLDAvis.display(lda_vis)

中国媒体关注的问题更为集中，港澳台·境外媒体关注的问题较为分散。